# Your first quantum game

We are going to make a game, so we'll need a game engine. We'll use a simple one made out of Jupyter notebooks.

In [ ]:
from jupyter_widget_engine import JupyterWidgetEngine

A minimal example of using this game engine is to simply set all the pixels to a certan colour.

In [ ]:
# set screen size
L = 8

# function called when setting up
def start(engine):
    # just move on to the first frame
    next_frame(engine)

# this is the function that does everything
def next_frame (engine):
    
    # remove initial text
    if engine.controller['A'].value:
        engine.screen['text'].set_text('')
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen[x,y].set_color('green')
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=L)

Now let's set one specific pixel to a different colour.

In [ ]:
# function called when setting up
def start(engine):
    
    # set a parameter to keep track of the player pixel
    engine.X = 1
    engine.Y = 2
    
    # then move on to the first frame
    next_frame(engine)

# this is the function that does everything
def next_frame (engine):
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen[x,y].set_color('green')
            
    # draw the player pixel
    engine.screen[engine.X,engine.Y].set_color('red')
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=L)

We'll move this around using the arrow buttons.

In [ ]:
# this is the function that does everything
def next_frame (engine):
    
    # remove initial text
    if engine.controller['A'].value:
        engine.screen['text'].set_text('')
    
    # change player position
    if engine.controller['up'].value:
        engine.Y -= 1
    if engine.controller['down'].value:
        engine.Y += 1
    if engine.controller['right'].value:
        engine.X += 1
    if engine.controller['left'].value:
        engine.X -= 1
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen[x,y].set_color('green')
            
    # draw the player pixel
    engine.screen[engine.X,engine.Y].set_color('red')
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=L)

Walking off the edge of the screen results in an error. We can fix this.

In [ ]:
# this is the function that does everything
def next_frame (engine):
    
    # remove initial text
    if engine.controller['A'].value:
        engine.screen['text'].set_text('')
    
    # change player position
    if engine.controller['up'].value:
        engine.Y -= 1
    if engine.controller['down'].value:
        engine.Y += 1
    if engine.controller['right'].value:
        engine.X += 1
    if engine.controller['left'].value:
        engine.X -= 1
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen[x,y].set_color('green')
            
    # draw the player pixel
    Xs = engine.X%engine.L
    Ys = engine.Y%engine.L
    engine.screen[Xs,Ys].set_color('red')
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=L)

Now let's move towards giving our pixel a more exciting world to explore. We'll use a function to decide what colour each point should be. We'll start by not changing very much.

In [ ]:
def get_color(X,Y):
    return 'green'

# this is the function that does everything
def next_frame (engine):
    
    # remove initial text
    if engine.controller['A'].value:
        engine.screen['text'].set_text('')
    
    # change player position
    if engine.controller['up'].value:
        engine.Y -= 1
    if engine.controller['down'].value:
        engine.Y += 1
    if engine.controller['right'].value:
        engine.X += 1
    if engine.controller['left'].value:
        engine.X -= 1
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            # get the 'world' coordinates X,Y from the onscreen coordinates x,y
            X = int(engine.X/engine.L)*L+x
            Y = int(engine.Y/L)*engine.L+y
            # set it to whatever colour it should be
            engine.screen[x,y].set_color(get_color(X,Y))
            
    # draw the player pixel
    Xs = engine.X%engine.L
    Ys = engine.Y%engine.L
    engine.screen[Xs,Ys].set_color('red')
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=L)

Now let's change `get_color` to create a beach.

In [ ]:
def get_color(X,Y):
    if X<12:
        color = 'green'
    else:
        color = 'orange'
    return color
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=L)

We'll now put a single qubit circuit inside this terrain generator, so that we can experiment with single qubit gates by making terrain. For that we'll need some Qiskit tools.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

First, let's see what a circuit looks like. Here's one with just a `ry` gate for a given angle of rotation.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

theta = 3.14159/2

qc = QuantumCircuit(1)
qc.ry(theta,0)

state = Statevector.from_instruction(qc)
probs = state.probabilities_dict()

print(probs)

Let's use the probaility of the output '1' as a height, and set the colour accordingly.

In [ ]:
def get_color(X,Y):
    
    # set an angle for ry based on the coordinate
    theta = (X+Y)*(3.14159/16)
    
    # create and simulate the circuit
    qc = QuantumCircuit(1)
    qc.ry(theta,0)
    state = Statevector.from_instruction(qc)
    probs = state.probabilities_dict()
    
    # get the prob of '1'
    if '1' in probs:
        height = probs['1']
    else:
        height = 0
    
    # set colour accordingly
    if height<0.1: # sea/river
        color = 'blue'
    elif height<0.3: # beach
        color = 'orange'
    elif height<0.9: # grass
        color = 'green'
    else: # mountain
        color = 'grey'
        
    return color
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=16)

Now experiment by making your own circuits to generate more interesting terrain. You might find [this](https://javafxpert.github.io/grok-bloch/) to be a handy tool to determine what different gates will do.